# ISPRS Part 2

This part is about plotting data with the DeepESDL platform. There are two main perspectives. First, most data formats from `xarray` can directly be plotted. Additionally, it is possible to setup the xcube viewer and use it as a very intuitive way to explore more complex raster datasets.

First: load some packages

In [ ]:
import xcube
from xcube.core.store import new_data_store
from xcube.webapi.viewer import Viewer
import matplotlib.pyplot as plt

## Data Loading

In this notebook we will only use the Earth system data cube (This is covered in Part 1. Here just see it as a blackbox).


In [ ]:
store = new_data_store("s3", 
                       root="deep-esdl-public", 
                       storage_options=dict(anon=True))
dataset = store.open_data('esdc-8d-0.25deg-256x128x128-3.0.1.zarr')


## Plotting with `xarray`

The `xarray` library is capable to plot data. All important libraries for this are already loaded with xcube in the DeepESDL platform. The plots are actually done by the famous `matplotlib` library based on the data. But you can also customize your plots.

### Simple 1D plots

First we want to concentrate on simple line plots and plots of one dimensional data. Mostly done for time series. For this we will have a look at the air temperatures in 2m height in Fremantle.
 This is a very simple plot and no customization is done. This technique is great for exploration.

In [ ]:
data = dataset['air_temperature_2m'].sel(
    lat='-32.05629322136265', lon = '115.74835460614544', method='nearest'
).sel(
    time = slice('2016-01-01','2021-12-31')
)

In [ ]:
data.plot()

The plot function allows a wide variety of inputs to control the graphics. Here, `matplotlib` is an extensive library to explore. The `xarray` package not just delivers the data, but also descriptions about the axes and so the axes labels are correct. Let's customize the plot a bit further:

- colors
- markers
- aspect ratio of the figure.

In [ ]:
data.plot(marker = 'o', color='green')
plt.title('2m air temperature in Fremantle from 2016 till 2021')

In [ ]:
data.plot(linestyle='--', marker = 'o', color='green', aspect=2, size=8)
plt.title('2m air temperature in Fremantle from 2016 till 2021')

### More 1D plots

There are many more kinds of plots for 1D data. And also there are ways to customize the plots even more. This is all based on the `matplotlib` library. So far we used `plot` without any specialization. We could have used as well: `dataset.plot.line()`. This is what is happening in the background anyway. But there are other possibilities as well.

In [ ]:
data.plot.hist()

Even more intersting is to bundle several plots together. You need to use slicing in more than one variable for this. The next plot shows the 2m air temperature at 3 different positions(Fremantle and 5 degrees east and westwards). This shows differences between ocean, coast and inland

In [ ]:
data = dataset['air_temperature_2m'].sel(
    lat='-32.0562', lon = ['110.7483', '115.7483', '120.7483'], method='nearest'
).sel(
    time = slice('2016-01-01','2021-12-31')
)

In [ ]:
data.plot()

Well this is not what we intended. In this case we need to specify the kind of plot as `line` our variable for the x-axis as well. This should be the `time` variable.
Please remember that those are just number experiments.

In [ ]:
ax = data.plot.line(x="time")
plt.legend(["Ocean", "Fremantle", "inland location"])

Well now we properly see the differences and how the ocean acts on 2m temperatures

### 2D plots

Two dimensional plots are very famous and with `xarray` they are also easy to create. Let's have a look at the 2m air temperature in Australia area 5 years ago (2019-10-22).

In [ ]:
WA_data = dataset.sel(
    lat = slice(-36, -13.0),
    lon = slice(111, 129)
).sel(
    time = '2019-10-22',
    method='nearest'
)
WA_data['air_temperature_2m'].plot()
plt.title("2m air temperature of Western Australia, around 2019-10-22")

We want to have a look at one more example. In case values are not available (`missing` or `NaN`), the `plot` function will actually care for this. Here is an example about this. The Gross Primary Productivity (GPP) is only available as data, where landmass is present. Also, there is no GPP in the desert region. Typically, this is used with the `viridis` colormap

In [ ]:
WA_data = dataset.sel(
    lat = slice(-36, -13.0),
    lon = slice(111, 129)
).sel(
    time = '2019-10-22',
    method='nearest'
)
WA_data['gross_primary_productivity'].plot(
    cmap='viridis'
)
plt.title("GPP of Western Australia, around 2019-10-22")

This is also the moment to show how to save such plots. For this you need to have the following command in the same cell as the plotting commands. Here we also introduce the possibility to change the colormap in the plot command.

In [ ]:
WA_data['gross_primary_productivity'].plot(
    cmap='viridis'
)
# Save the figure
plt.savefig('GPP_WA_2019_10_24.pdf')

## xcube viewer

As part of the DeepESDL platform, there is one more feature for plotting. This is the xcube viewer. It is a whole dynamic and interactive plotting experience for datacubes. We want to concentrate our view to the central african region between 2015 and 2020. Therefore we need to create a dataset first.

In [ ]:
# first limit to variables
gpp_data = dataset[
    ['gross_primary_productivity']
]
# Now limit to spatial and temporal dimensions
final_WA_data = gpp_data.sel(
    lat = slice(-36, -13.0),
    lon = slice(111, 129)
).sel(
    time = slice('2020-01-01', '2020-12-31')
)
final_WA_data

This datacube is still to big to find a suitable way for plotting. But the xcube viewer can be a solution.

*Please set up a scaling according to the data to get the best experience. You can do this with clicking the colorbar.*

Recommended: viridis colorbar from 0 to 3.5

In [ ]:
viewer = Viewer()
viewer.add_dataset(final_WA_data)
viewer.show()